# Imported Modules 

In [1]:
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import requests
from sqlalchemy import create_engine
import json
from datetime import datetime
import time
from time import sleep
import sqlite3

# Get State Information

In [2]:
# Use Zone ID for USA to get the list of states
Northern_America = [601,602,603,604,605,606]

In [9]:
# Loop through the API call to get the state name, slug, and assigned zone number 
state_slug = []
state_name = []
zone_number = []


for zone in Northern_America:
    url = f"https://trefle.io/api/v1/distributions/{zone}?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0"
    print("downloading: " + url)
    response = requests.get(url)
    data = response.json()['data']['children']
    
    for state in data:
        state_slug.append(state['slug'])
        state_name.append(state['name'])
        zone = zone
        zone_number.append(zone)
        

downloading: https://trefle.io/api/v1/distributions/601?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/602?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/603?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/604?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/605?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/606?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0


In [10]:
# To create a complete understanding of the US Map. We need to know the zone name as well
zone_name = []
zone_id = []
for zone in Northern_America:
    url = f"https://trefle.io/api/v1/distributions/{zone}?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0"
    print("downloading: " + url)
    response = requests.get(url)
    data = response.json()['data']['name']
    zone = zone
    zone_id.append(zone)
    zone_name.append(data)
    

downloading: https://trefle.io/api/v1/distributions/601?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/602?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/603?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/604?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/605?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0
downloading: https://trefle.io/api/v1/distributions/606?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0


In [11]:
# create a DF that will be merge with the complete list 
# Data will be merged on zone_id: to show which state belongs to which zone
zone_info = pd.DataFrame({"zone_id": zone_id, "zone_name": zone_name})
zone_info

,zone_id,zone_name
0,601,Northwestern U.S.A.
1,602,North-Central U.S.A.
2,603,Northeastern U.S.A.
3,604,Southwestern U.S.A.
4,605,South-Central U.S.A.
5,606,Southeastern U.S.A.


In [16]:
# Create a DF for all States (exception Alaska, Hawaii)
state_df = pd.DataFrame({"state_slug": state_slug, "state_name": state_name,"zone_id": zone_number})
state_df.head()

,state_slug,state_name,zone_id
0,was,Washington,601
1,wyo,Wyoming,601
2,ida,Idaho,601
3,mnt,Montana,601
4,col,Colorado,601


In [17]:
# Merge the DF into one that displays State info and zone name
state_merge = state_df.merge(zone_info, how='inner', on='zone_id')
state_merge.head()

,state_slug,state_name,zone_id,zone_name
0,was,Washington,601,Northwestern U.S.A.
1,wyo,Wyoming,601,Northwestern U.S.A.
2,ida,Idaho,601,Northwestern U.S.A.
3,mnt,Montana,601,Northwestern U.S.A.
4,col,Colorado,601,Northwestern U.S.A.


In [19]:
# list of all the plant species found in US with state_slug
all_state = pd.read_csv("full_state.csv")

In [21]:
all_info = all_state.merge(state_merge,how="inner", on="state_slug")
all_info.head()

,species_common_name,species_family,image_url,species_slug,discovery_year,species_scntfc_name,species_fmly_cmmn_name,state_slug,state_name,zone_id,zone_name
0,Stinging nettle,Urticaceae,https://bs.floristic.org/image/o/85256a1c2c098...,urtica-dioica,1753.0,Urtica dioica,Nettle family,was,Washington,601,Northwestern U.S.A.
1,Orchardgrass,Poaceae,https://bs.floristic.org/image/o/428f40dadfa02...,dactylis-glomerata,1753.0,Dactylis glomerata,Grass family,was,Washington,601,Northwestern U.S.A.
2,Narrowleaf plantain,Plantaginaceae,https://bs.floristic.org/image/o/78a8374f009e6...,plantago-lanceolata,1753.0,Plantago lanceolata,Plantain family,was,Washington,601,Northwestern U.S.A.
3,Red fescue,Poaceae,https://bs.floristic.org/image/o/0b932c8a275ef...,festuca-rubra,1753.0,Festuca rubra,Grass family,was,Washington,601,Northwestern U.S.A.
4,Creeping buttercup,Ranunculaceae,https://bs.floristic.org/image/o/c6d9a5222b6ef...,ranunculus-repens,1753.0,Ranunculus repens,Buttercup family,was,Washington,601,Northwestern U.S.A.


In [22]:
# converting year from float to string before as sqlite DB
all_info['discovery_year'] = all_info['discovery_year'].astype(str).replace('\.0','', regex=True)
all_info.head()

,species_common_name,species_family,image_url,species_slug,discovery_year,species_scntfc_name,species_fmly_cmmn_name,state_slug,state_name,zone_id,zone_name
0,Stinging nettle,Urticaceae,https://bs.floristic.org/image/o/85256a1c2c098...,urtica-dioica,1753,Urtica dioica,Nettle family,was,Washington,601,Northwestern U.S.A.
1,Orchardgrass,Poaceae,https://bs.floristic.org/image/o/428f40dadfa02...,dactylis-glomerata,1753,Dactylis glomerata,Grass family,was,Washington,601,Northwestern U.S.A.
2,Narrowleaf plantain,Plantaginaceae,https://bs.floristic.org/image/o/78a8374f009e6...,plantago-lanceolata,1753,Plantago lanceolata,Plantain family,was,Washington,601,Northwestern U.S.A.
3,Red fescue,Poaceae,https://bs.floristic.org/image/o/0b932c8a275ef...,festuca-rubra,1753,Festuca rubra,Grass family,was,Washington,601,Northwestern U.S.A.
4,Creeping buttercup,Ranunculaceae,https://bs.floristic.org/image/o/c6d9a5222b6ef...,ranunculus-repens,1753,Ranunculus repens,Buttercup family,was,Washington,601,Northwestern U.S.A.


# -------------------------------------------------------------------------------------

# Data Scraping through multiple Pages for Each State in the list to Populate Data on Species of Plants Per State

In [ ]:
# We created the list of state slug from the first script
state_slug

# empty list where the data will append to from the for loop
pl_name = []
pl_family = []
pl_img = []
pl_slug = []
pl_yr = []
pl_sct_name = []
pl_family_common_name = []
slug_x = []

# Loop through each state to get the API Link for List of species found in the state
for slug in state_slug:
    url = f"https://trefle.io/api/v1/distributions/{slug}?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0"
    response = requests.get(url)
    data = response.json()['data']['links']['species']
# Print to check which step of the script is currently running. Also to trouble shoot if script breaks at location
    print(f"-------------------------------------")
    print(f"downloading: '{slug}' data")
    
#  Makes a API request based on the link collected from first request
    url = f"https://trefle.io{data}?token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0"
    response = requests.get(url)
    meta_data = response.json()['meta']['total']
    
# Using the meta data value, we calculated the total amount of pagination per state species list
    if meta_data %20 != 0:
#         realized that rounding the value was making us miss the value due to 0.5 rounding  
        pages = meta_data/20 + 0.5 
        pages = round(pages)
    else:
        pages = int(meta_data/20)

    pages = int(pages)
    print(pages)

# Create a For loop that would make an API call for range of pages for each state species list     
    for page_num in range(1,pages + 1):
        url = f"https://trefle.io/api/v1/species?page={page_num}&zone_id={slug}&token=lJB461ruHXGiw6SBUD_liGd6XLjQI6Kyqj0AGLhhKn0"
        
# Print to check which step of the script is currently running. Also to trouble shoot if script breaks at location
        print(f"I am on page: {page_num}")
        print(url)
        response = requests.get(url)
        p_data = response.json()['data']
        print(f"downloading data for {slug} on page {page_num}")
        startTime = time.time()
        for i in range(0,2):
#            print(i)
           # making delay for 1 second
           time.sleep(1)
        endTime = time.time()
        elapsedTime = endTime - startTime
        print("Elapsed Time = %s" % elapsedTime)
        for plant in p_data:
            plant_name = plant['common_name']
            plant_family = plant['family']
            plant_img = plant['image_url']
            plant_slug = plant['slug']
            plant_year = plant['year']
            plant_science = plant['scientific_name']
            plant_family_common_name = plant['family_common_name']
            slug = slug
            slug_x.append(slug)
            pl_name.append(plant_name)
            pl_family.append(plant_family)
            pl_img.append(plant_img)
            pl_slug.append(plant_slug)
            pl_yr.append(plant_year)
            pl_sct_name.append(plant_science)
            pl_family_common_name.append(plant_family_common_name)
            
            
        


In [ ]:
# create a DF from the lists 
Species_detail = pd.DataFrame({"species_common_name": pl_name,
                                      "species_family": pl_family,
                                      "image_url": pl_img,
                                       "species_slug": pl_slug,
                                      "discovery_year": pl_yr,
                                      "species_scntfc_name":pl_sct_name,
                                      "species_fmly_cmmn_name": pl_family_common_name,
                                      "state_slug": slug_x})

# Save the DF as SQLITE DB

In [23]:
# create a connection with SQLite 
conn = sqlite3.connect('plant_US.sqlite')

conn.cursor().execute("create table if not exists US_species ({})".format(','.join(all_info.columns)))

In [24]:
# Append the rows from DF into the SQL table 
for row in all_info.iterrows():
    sql = 'insert into US_species ({}) values ({})'.format(','.join(all_info.columns),','.join(['?']*len(all_info.columns)))
    conn.cursor().execute(sql,tuple(row[1]))
    
conn.commit()

In [45]:
# create a connection with SQLite 
conn = sqlite3.connect('species_US.sqlite')

conn.cursor().execute("create table if not exists species_db ({})".format(','.join(Species_detail.columns)))


1022

In [42]:
# Append the rows from DF into the SQL table 
for row in Species_detail.iterrows():
    sql = 'insert into species_db ({}) values ({})'.format(','.join(Species_detail.columns),','.join(['?']*len(Species_detail.columns)))
    conn.cursor().execute(sql,tuple(row[1]))
    
conn.commit()

1510.75